# Group 17 Proposal (Weekend Airbnb in Paris)


## Inrtoduction 







Airbnbs have specific attributes (room type, cleanliness rating, superhost status etc.) that determine its price and perceived quality. The quality of a Airbnbs often evaluated by users through a online/digital rating system. Attributes of Parisian Airbnbs have been determined and collected into a data set that is available on kaggle. It can be used to analyze trends in Airbnb prices and popularity across different cities and neighborhoods, as well as to identify factors that may influence prices and demand. The data set includes 19 variables in each respective column, with each row documenting an Airbnb's price. 

The following are the potential predictor variables including their description: 

| Column name | Description |
| --- | --- |
| realSum | The total price of the Airbnb listing. (Numeric) |
| room_type | The type of room being offered (e.g. private, shared, etc.). (Categorical) |
| room_shared | Whether the room is shared or not. (Boolean) |
| room_private | Whether the room is private or not. (Boolean) |
| person_capacity | The maximum number of people that can stay in the room. (Numeric) |
| host_is_superhost | Whether the host is a superhost or not. (Boolean) |
| multi | Whether the listing is for multiple rooms or not. (Boolean) |
| biz | Whether the listing is for business purposes or not. (Boolean) |
| cleanliness_rating | The cleanliness rating of the listing. (Numeric) |
| guest_satisfaction_overall | The overall guest satisfaction rating of the listing. (Numeric) |
| bedrooms | The number of bedrooms in the listing. (Numeric) |
| dist | The distance from the city centre. (Numeric) |
| metro_dist | The distance from the nearest metro station. (Numeric) |
| lng | The longitude of the listing. (Numeric) |
| lat | The latitude of the listing. (Numeric) |

This project will take that data and attempt to answer the question: What factors affect Airbnb Prices in Paris on Weekends?

In [42]:
#load the necessary packages
library(tidyverse)
library(repr)
library(tidymodels)
library(gridExtra)
options(repr.matrix.max.rows = 6)

## Preliminary exploratory data analysis

In [62]:
# read data frame from kaggle; doesn't seem to work
# paris_data_link <- "https://www.kaggle.com/datasets/thedevastator/airbnb-prices-in-european-cities?select=paris_weekends.csv"
# # use ';' delimiter
# paris_data <- read_csv(paris_data_link, sep = ",")
# paris_data


#old_code (that works): 
paris_data <- read_csv("paris_weekends.csv")
paris_data

New names:
• `` -> `...1`
Rows: 3558 Columns: 20
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): room_type
dbl (16): ...1, realSum, person_capacity, multi, biz, cleanliness_rating, gu...
lgl  (3): room_shared, room_private, host_is_superhost

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


...1,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
<dbl>,<dbl>,<chr>,<lgl>,<lgl>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,536.3967,Entire home/apt,FALSE,FALSE,5,FALSE,0,1,9,89,1,1.3512012,0.2123455,390.7768,19.00155,1030.739,47.55037,2.35900,48.86800
1,290.1016,Private room,FALSE,TRUE,2,TRUE,0,0,10,97,1,0.6998212,0.1937103,518.4783,25.21104,1218.659,56.21958,2.35385,48.86282
2,445.7545,Entire home/apt,FALSE,FALSE,4,FALSE,0,1,10,100,1,0.9689817,0.2943429,432.6899,21.03958,1069.895,49.35674,2.36023,48.86375
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
3555,223.9258,Entire home/apt,FALSE,FALSE,2,FALSE,1,0,9,89,1,4.205205,0.2530289,172.6589,8.395552,406.5859,18.75676,2.40100,48.87700
3556,200.8575,Entire home/apt,FALSE,FALSE,2,TRUE,0,0,9,93,1,2.891214,0.2406744,235.1679,11.435057,602.4517,27.79250,2.38200,48.87400
3557,301.2862,Entire home/apt,FALSE,FALSE,2,FALSE,0,0,10,92,1,3.469749,0.5085167,200.6376,9.756019,478.9904,22.09694,2.39992,48.85886


In [44]:
#filter the needed columns
new_paris_data <- paris_data |>
  select(realSum, cleanliness_rating, guest_satisfaction_overall, dist, metro_dist, 
        room_type, host_is_superhost)
new_paris_data 

realSum,cleanliness_rating,guest_satisfaction_overall,dist,metro_dist,room_type,host_is_superhost
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>
536.3967,9,89,1.3512012,0.2123455,Entire home/apt,FALSE
290.1016,10,97,0.6998212,0.1937103,Private room,TRUE
445.7545,10,100,0.9689817,0.2943429,Entire home/apt,FALSE
⋮,⋮,⋮,⋮,⋮,⋮,⋮
223.9258,9,89,4.205205,0.2530289,Entire home/apt,FALSE
200.8575,9,93,2.891214,0.2406744,Entire home/apt,TRUE
301.2862,10,92,3.469749,0.5085167,Entire home/apt,FALSE


In [45]:
#set the seed
set.seed(8888)

In [46]:
#splitting the data
paris_split <- initial_split(new_paris_data, prop = 0.75, strata = realSum) 
paris_train <- training(paris_split)
paris_test <- testing(paris_split)

In [47]:
#Table and Counts
#Cleanliness_rating
paris_proportions1 <- paris_train |> #use TRAINING data
group_by(cleanliness_rating) |>
summarize(n = n())  #used to count the number of observations in a given group
paris_proportions1

#guest_satisfaction_overall
paris_proportions2 <- paris_train |>
group_by(guest_satisfaction_overall) |>
summarize(n = n())
paris_proportions2

#dist
paris_proportions3 <- paris_train |>
group_by(dist) |>
summarize(n = n()) 
paris_proportions3

#metro_dist
paris_proportions4 <- paris_train |>
group_by(metro_dist) |>
summarize(n = n()) 
paris_proportions4

#room_type
paris_proportions5 <- paris_train |>
group_by(room_type) |>
summarize(n = n()) 
paris_proportions5

#host_is_superhost
paris_proportions6 <- paris_train |>
group_by(host_is_superhost) |>
summarize(n = n()) 
paris_proportions6


cleanliness_rating,n
<dbl>,<int>
2,3
4,7
5,2
⋮,⋮
8,272
9,947
10,1334


guest_satisfaction_overall,n
<dbl>,<int>
20,4
40,5
50,4
⋮,⋮
98,138
99,70
100,584


dist,n
<dbl>,<int>
0.1395432,1
0.1470952,1
0.1714837,1
⋮,⋮
7.620912,1
7.680000,1
7.703733,1


metro_dist,n
<dbl>,<int>
0.003220008,1
0.003935058,1
0.006388847,1
⋮,⋮
0.9102045,1
0.9689053,1
1.0458365,1


room_type,n
<chr>,<int>
Entire home/apt,2065
Private room,568
Shared room,34


host_is_superhost,n
<lgl>,<int>
FALSE,2290
TRUE,377


In [61]:
#Mean 
mean1 <- paris_train |> #use TRAINING data 
    select(cleanliness_rating:metro_dist)|> 
    map_df(max, na.rm = TRUE)
mean1

cleanliness_rating,guest_satisfaction_overall,dist,metro_dist
<dbl>,<dbl>,<dbl>,<dbl>
10,100,7.703733,1.045836
